In [1]:

import numpy as np
import torch
from datasets import load_from_disk
from tashaphyne.stemming import ArabicLightStemmer
from transformers import AutoTokenizer, AutoModelForMaskedLM

from processingDatasetDotless import *

In [2]:
dataset = load_from_disk("/Users/ammar/Developer/git-repos/dotless/Models/Model v3/AR-multi-dotted-Small-arrow")


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Masked', 'Options', 'Target'],
        num_rows: 500000
    })
})

In [4]:
dataset['train'][20]

{'Masked': 'يوسف الذين دفنوا السلاح و المهمات العسكرية في [MASK] من جب بالعاصمة',
 'Options': ['آكثر',
  'آكثز',
  'آكتر',
  'آكتز',
  'آكبر',
  'آكبز',
  'إكثر',
  'إكثز',
  'إكتر',
  'إكتز',
  'إكبر',
  'إكبز',
  'أكثر',
  'أكثز',
  'أكتر',
  'أكتز',
  'أكبر',
  'أكبز',
  'اكثر',
  'اكثز',
  'اكتر',
  'اكتز',
  'اكبر',
  'اكبز'],
 'Target': 'أكثر'}

In [5]:
indicies = []
for i in range(10000):
    length = len(dataset['train'][i]["Options"])
    if length > 10 and length <= 20:
        indicies.append(i)

print(str(indicies[0:5]) + "..." + str(indicies[-5:]))
print(len(indicies))
dataset['train'][indicies[3]]


[13, 14, 28, 41, 57]...[9968, 9991, 9993, 9996, 9997]
1306


{'Masked': 'السؤال كيف توفق بين دخلك والمنصرفات [MASK] الله كريم فخلص إلى أن',
 'Options': ['ؤةي',
  'ؤةى',
  'ؤةئ',
  'ؤهي',
  'ؤهى',
  'ؤهئ',
  'وةي',
  'وةى',
  'وةئ',
  'وهي',
  'وهى',
  'وهئ'],
 'Target': 'وهي'}

In [6]:
with open("/Users/ammar/Developer/git-repos/dotless/Models/Model v4/roots.txt", 'r') as file:
    rootlist = set(file.read().split())


In [7]:
with open("/Users/ammar/Developer/git-repos/dotless/data/arwiki.wordlist", 'r') as file:
    arDictionary = set(file.read().split())


In [8]:
def calculate_len_variations(word):
    total_variations = 1

    for char in word:
        total_variations *= len(iLetters[char])

    return total_variations

In [9]:
iletter_set = set(iLetters.keys())


In [10]:

def generate_random_word(length):
    return ''.join(random.choice(list(iletter_set)) for _ in range(length))

while True:
    random_word = generate_random_word(10) # You can adjust the length as needed
    num_variations = calculate_len_variations(random_word)
    if 50_000 < num_variations < 60_000:
        print("Found a word with desired variations:")
        print("Word:", random_word)
        print("Number of variations:", num_variations)
        break

Found a word with desired variations:
Word: ىااوىحطاده
Number of variations: 54000


In [11]:
word = "ارىطاصصححا"
print(len(word))
calculate_len_variations(parse_dotless_text(word))

10


54000

In [12]:
variations = allVariation(parse_dotless_text(word))
print(variations)
print(len(variations))

['ءزئظءضضخخا', 'ءزئظءضضخخأ', 'ءزئظءضضخخإ', 'ءزئظءضضخخآ', 'ءزئظءضضخخء', 'ءزئظءضضخحا', 'ءزئظءضضخحأ', 'ءزئظءضضخحإ', 'ءزئظءضضخحآ', 'ءزئظءضضخحء', 'ءزئظءضضخجا', 'ءزئظءضضخجأ', 'ءزئظءضضخجإ', 'ءزئظءضضخجآ', 'ءزئظءضضخجء', 'ءزئظءضضحخا', 'ءزئظءضضحخأ', 'ءزئظءضضحخإ', 'ءزئظءضضحخآ', 'ءزئظءضضحخء', 'ءزئظءضضححا', 'ءزئظءضضححأ', 'ءزئظءضضححإ', 'ءزئظءضضححآ', 'ءزئظءضضححء', 'ءزئظءضضحجا', 'ءزئظءضضحجأ', 'ءزئظءضضحجإ', 'ءزئظءضضحجآ', 'ءزئظءضضحجء', 'ءزئظءضضجخا', 'ءزئظءضضجخأ', 'ءزئظءضضجخإ', 'ءزئظءضضجخآ', 'ءزئظءضضجخء', 'ءزئظءضضجحا', 'ءزئظءضضجحأ', 'ءزئظءضضجحإ', 'ءزئظءضضجحآ', 'ءزئظءضضجحء', 'ءزئظءضضججا', 'ءزئظءضضججأ', 'ءزئظءضضججإ', 'ءزئظءضضججآ', 'ءزئظءضضججء', 'ءزئظءضصخخا', 'ءزئظءضصخخأ', 'ءزئظءضصخخإ', 'ءزئظءضصخخآ', 'ءزئظءضصخخء', 'ءزئظءضصخحا', 'ءزئظءضصخحأ', 'ءزئظءضصخحإ', 'ءزئظءضصخحآ', 'ءزئظءضصخحء', 'ءزئظءضصخجا', 'ءزئظءضصخجأ', 'ءزئظءضصخجإ', 'ءزئظءضصخجآ', 'ءزئظءضصخجء', 'ءزئظءضصحخا', 'ءزئظءضصحخأ', 'ءزئظءضصحخإ', 'ءزئظءضصحخآ', 'ءزئظءضصحخء', 'ءزئظءضصححا', 'ءزئظءضصححأ', 'ءزئظءضصححإ', 'ءزئظءضصححآ', 'ءزئظءضصححء', 'ءزئظءضصحجا', 'ءزئظ

In [13]:
"#" in list(rootlist)

False

In [14]:
model_name = "CAMeL-Lab/bert-base-arabic-camelbert-mix"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']
[1, 3, 0, 2, 4]


In [16]:
def mask_word(text, index=None):
    words = text.split()
    if index == None: index = random.randint(0, len(words) - 1)

    for i, word in enumerate(words):
        if i == index:
            word_to_replace = words[i]
            variations = allVariation(parse_dotless_text(word_to_replace))
            words[i] = "[MASK]"
            modified_string = ' '.join(words)
            masked = (modified_string)
            options = (variations)
            targets = (word_to_replace)

    return {"Masked": masked, "Options": options, "Target": targets}

In [17]:
ArListem = ArabicLightStemmer()


In [18]:
word = u'السعادة'

In [19]:
tokenized_word = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(word))
tokenized_word

[13876]

In [20]:
word_tokenID = tokenizer.convert_ids_to_tokens(tokenized_word)
word_tokenID

['السعادة']

print(word_tokenID[0] + word_tokenID[1])

In [21]:
stem = ArListem.light_stem(word_tokenID[0])
root = ArListem.get_root()
print("stem:", stem)
print("root:", root)

stem: سعاد
root: سعد


In [22]:
stem = ArListem.light_stem(word)
root = ArListem.get_root()
print("stem:", stem)
print("root:", root)

stem: سعاد
root: سعد


In [23]:
root in rootlist

True

In [24]:
sentance = "جمع [MASK] سكين"
word1 ="السعادة" 
word2 = "##ة"


tokenized_text = tokenizer.tokenize(sentance)
masked_word_index = tokenized_text.index('[MASK]')
input_ids = tokenizer.convert_tokens_to_ids(tokenized_text)
input_ids = torch.tensor(input_ids, dtype=torch.long).unsqueeze_(0)

print(tokenized_text)
print(input_ids)

with torch.no_grad():
    output = model(input_ids)

predictions = output.logits[0, masked_word_index].softmax(dim=0)

print(word1, predictions[tokenizer.convert_tokens_to_ids(word1)].item())
print(word2, predictions[tokenizer.convert_tokens_to_ids(word2)].item())


['جمع', '[MASK]', 'سكين']
tensor([[ 3341,     4, 29555]])
السعادة 5.7757379323675195e-08
##ة 2.4179865931728273e-07


In [25]:
word_tokenID = [w.replace('#', '') for w in word_tokenID]
print(word_tokenID)

roots = []
for w in word_tokenID:
    stem = ArListem.light_stem(w)
    root = ArListem.get_root()
    roots.append(root)

print(roots)

is_root = [w in rootlist for w in roots]
print(is_root)

#check if there is at least one true
true_root = any(is_root)
print(true_root)

['السعادة']
['سعد']
[True]
True


In [26]:
def check_for_root(tokens):
    tokens = [w.replace('#', '') for w in tokens]
    roots = []
    for w in tokens:
        stem = ArListem.light_stem(w)
        root = ArListem.get_root()
        roots.append(root)

    is_roots = [w in rootlist for w in roots]
    any_true_root = any(is_roots)
    all_true_root = all(is_roots)
    real_roots = [roots[i] for i in range(len(roots)) if is_roots[i]]
    return any_true_root, all_true_root, is_roots, roots, real_roots

In [27]:
tokenizer.tokenize("السعادة")

['السعادة']

In [28]:
check_for_root(["السعادة"])

(True, True, [True], ['سعد'], ['سعد'])

text = "The Milky Way is a [MASK] galaxy."


tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
inputs = tokenizer(text, return_tensors="pt")
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")
logits = model(**inputs).logits
mask_token_logits = logits[0, mask_token_index, :]

top_tokens = torch.topk(mask_token_logits, 10000, dim=1).indices[0].tolist()[-10:-1]

for token in top_tokens:
    token_text = tokenizer.convert_ids_to_tokens(token)
    print(token_text)


In [29]:


def generate_probabilties_end(example):
    input_text = example["Masked"]
    candidates = example["Options"]

    word_probabilities = get_candidate_word_probabilities(input_text, candidates)

    sorted_words = sorted(word_probabilities, key=word_probabilities.get, reverse=True)
    if len(sorted_words) > 0:
        most_probable_word = sorted_words[0]
    else:
        most_probable_word = None
        #print(example["Target"])

    return word_probabilities, sorted_words, most_probable_word


In [30]:

def get_candidate_word_probabilities(input_text, candidate_words):
    tokenized_text = tokenizer.tokenize(input_text)
    masked_word_index = tokenized_text.index('[MASK]')
    input_ids = tokenizer.convert_tokens_to_ids(tokenized_text)
    input_ids = torch.tensor(input_ids, dtype=torch.long).unsqueeze_(0)

    with torch.no_grad():
        output = model(input_ids)

    predictions = output.logits[0, masked_word_index].softmax(dim=0)
    initial_length = len(candidate_words)
    
    filtered_candidate_probabilities = []
    filtered_candidate_words = []
    
    for word in candidate_words:
        
        word_tokens = tokenizer.tokenize(word)
        any_true_root, all_true_root, is_roots, roots, real_roots = check_for_root(word_tokens)
            
        # print("word:", word)
        # print("true_root:", true_root)
        # print("is_roots:", is_roots)
        # print("roots:", roots)
        # print("word_tokens:", word_tokens)
        # print("-------------------------------------")
        
        if not true_root:
            # print("Rejected")
            # print("--------------------------------------------------------------------------")
            continue
            
        word_ids = tokenizer.convert_tokens_to_ids(word_tokens)
        
        total_predictions = 0
        predictionList = []
        for wid in word_ids:
            prediction = predictions[wid].item()
            predictionList.append(prediction)
            total_predictions += prediction     
        average = sum(predictionList) / len(predictionList)

        filtered_candidate_probabilities.append(average)
        filtered_candidate_words.append(word)
        # print(f"{word} : {average:.20f}")
        # print({token: probability for token, probability in zip(word_tokens, predictionList)})
        # print("-------------------------------------")
    
    candidate_probabilities = {word: probability for word, probability in zip(filtered_candidate_words, filtered_candidate_probabilities)}
    

    return candidate_probabilities

In [31]:

def get_candidate_word_probabilities(input_text, candidate_words):
    tokenized_text = tokenizer.tokenize(input_text)
    masked_word_index = tokenized_text.index('[MASK]')
    input_ids = tokenizer.convert_tokens_to_ids(tokenized_text)
    input_ids = torch.tensor(input_ids, dtype=torch.long).unsqueeze_(0)

    with torch.no_grad():
        output = model(input_ids)

    predictions = output.logits[0, masked_word_index].softmax(dim=0)

    
    # Tokenize and convert all candidate words to ids at once
     
    pre_tokenized_candidate_words = [tokenizer.tokenize(word) for word in candidate_words]
    #print(tokenized_candidate_words)
    
    tokenized_candidate_words = []
    verified_words = []
    for word in pre_tokenized_candidate_words:
        rep_word = candidate_words[pre_tokenized_candidate_words.index(word)]
        if len(word) == 1:
            tokenized_candidate_words.append(word[0])
            verified_words.append(word[0])
        # else:
        #     any_true_root, all_true_root, is_roots, roots, real_roots = check_for_root(word)
            
        #     if true_root:
        #         longest_root = max(real_roots, key=len)
        #         tokenized_candidate_words.append(longest_root)
        #         verified_words.append(rep_word)
                
        #         print("word:", rep_word)
        #         print("tokenized_word", word)
        #         print("is_roots:", is_roots)
        #         print("roots:", roots)
        #         print("longest_root:", longest_root)
        #         print("--------------------------------------")
            
    #print(tokenized_candidate_words)
    #print(verified_words)
    flattened_tokenized_candidate_words = [word for sublist in tokenized_candidate_words for word in sublist]
    #print(flattened_tokenized_candidate_words)

    candidate_word_ids = [tokenizer.convert_tokens_to_ids(TCW) for TCW in flattened_tokenized_candidate_words]
    #print(candidate_word_ids)

    # Use a list comprehension to populate the dictionary
    candidate_probabilities = {word: predictions[word_id].item() for word, word_id in
                               zip(verified_words, candidate_word_ids)}
    #print(candidate_probabilities)

    return candidate_probabilities

In [32]:
num_eg = 0
#example = dataset['train'][indicies[num_eg]]
example = mask_word("السعادة تجلب السرور", 0)
word_probabilities, sorted_words, most_probable_word = generate_probabilties_end(example)
print("Length of words:", len(sorted_words))
for word in sorted_words:
    probability = word_probabilities[word]
    print(f"Word: '{word}', Probability: {probability:.10f}")

print()

print("Most probable word:", most_probable_word)
print("Target word:", example["Target"])
print("------------------------------------------")

found = False
for i in range(len(sorted_words)):
    if sorted_words[i] == example["Target"]:
        print("Sucess at probability level:", i)
        found = True
        break
if not found: print("Not found.")
sucess_level = i

example

Length of words: 2
Word: 'السعادة', Probability: 0.0010324180
Word: 'السعاده', Probability: 0.0003179799

Most probable word: السعادة
Target word: السعادة
------------------------------------------
Sucess at probability level: 0


{'Masked': '[MASK] تجلب السرور',
 'Options': ['ءلشغءذه',
  'ءلشغءذة',
  'ءلشغءده',
  'ءلشغءدة',
  'ءلشغآذه',
  'ءلشغآذة',
  'ءلشغآده',
  'ءلشغآدة',
  'ءلشغإذه',
  'ءلشغإذة',
  'ءلشغإده',
  'ءلشغإدة',
  'ءلشغأذه',
  'ءلشغأذة',
  'ءلشغأده',
  'ءلشغأدة',
  'ءلشغاذه',
  'ءلشغاذة',
  'ءلشغاده',
  'ءلشغادة',
  'ءلشعءذه',
  'ءلشعءذة',
  'ءلشعءده',
  'ءلشعءدة',
  'ءلشعآذه',
  'ءلشعآذة',
  'ءلشعآده',
  'ءلشعآدة',
  'ءلشعإذه',
  'ءلشعإذة',
  'ءلشعإده',
  'ءلشعإدة',
  'ءلشعأذه',
  'ءلشعأذة',
  'ءلشعأده',
  'ءلشعأدة',
  'ءلشعاذه',
  'ءلشعاذة',
  'ءلشعاده',
  'ءلشعادة',
  'ءلسغءذه',
  'ءلسغءذة',
  'ءلسغءده',
  'ءلسغءدة',
  'ءلسغآذه',
  'ءلسغآذة',
  'ءلسغآده',
  'ءلسغآدة',
  'ءلسغإذه',
  'ءلسغإذة',
  'ءلسغإده',
  'ءلسغإدة',
  'ءلسغأذه',
  'ءلسغأذة',
  'ءلسغأده',
  'ءلسغأدة',
  'ءلسغاذه',
  'ءلسغاذة',
  'ءلسغاده',
  'ءلسغادة',
  'ءلسعءذه',
  'ءلسعءذة',
  'ءلسعءده',
  'ءلسعءدة',
  'ءلسعآذه',
  'ءلسعآذة',
  'ءلسعآده',
  'ءلسعآدة',
  'ءلسعإذه',
  'ءلسعإذة',
  'ءلسعإده',
  'ءلسعإدة',
  'ءلسعأذه',
  'ءلسع

In [33]:

count = np.zeros(1000000, dtype=int)
#longWords = []
#veryLongWords = []
#veryVeryLongWords = []
unknownWords = []
start = 0
end = 101
iterationCount = end - start - 1
for i in range(start, end):
    num_eg = i
    #example1 = mask_random_word("التحدي الصعب يكمن في فهم التفاصيل الدقيقة لهذه الفكرة المعقدة")
    #example1 = dataset['train'][indicies[num_eg]]
    example1 = dataset['train'][num_eg]

    word_probabilities, sorted_words, most_probable_word = generate_probabilties_end(example1)
    if most_probable_word == None:
        unknownWords.append(example1["Target"])
        continue
    
    for word in sorted_words:
        probability = word_probabilities[word]

    #if len(sorted_words) >= 500 and len(sorted_words) < 1000: longWords.append(example1["Target"])
    #elif len(sorted_words) >= 1000 and len(sorted_words) < 100000: veryLongWords.append(example1["Target"])
    #elif len(sorted_words) >= 100000 and len(sorted_words) <1000000: veryVeryLongWords.append(example1["Target"])
    #elif len(sorted_words) >= 1000000: veryVeryLongWords.append(example1["Target"])

    for j in range(len(sorted_words)):
        if sorted_words[j] == example1["Target"]:
            count[j] += 1
            break

for k in range(len(count)):
    numberCount = count[k]
    if numberCount != 0:
        print(numberCount, "placments at level", k, "  \t", (numberCount / iterationCount) * 100, "%")

print()
print("Number of examples tested:", iterationCount)
print("Number of examples that yielded a result:", np.sum(count))


57 placments at level 0   	 56.99999999999999 %
14 placments at level 1   	 14.000000000000002 %
6 placments at level 2   	 6.0 %

Number of examples tested: 100
Number of examples that yielded a result: 77


In [34]:
print(len(rootlist))
print(len(arDictionary))

9520
1698379


In [35]:
knownUnknownWords = []
stillUnknownWords = []
for word in unknownWords:
    if word in arDictionary:
        knownUnknownWords.append(word)
    else:
        stillUnknownWords.append(word)

print("knownUnknownWords:", knownUnknownWords)
print("stillUnknownWords:", stillUnknownWords)

print(len(knownUnknownWords))
print(len(stillUnknownWords))

knownUnknownWords: ['الزملا', 'تزينت', 'تجربتها', 'بنجران', 'بوشناق', 'ودلائل', 'ناروتو', 'الفندقية', 'المئوية', 'التصفية', 'أدخله', 'المتلاحقة', 'بالرباط', 'لمنهج', 'لآثار', 'وشعرا', 'إخطاره', 'والسياح', 'ذويه', 'المطرقة']
stillUnknownWords: ['نيئن', 'المزيدكشفت', 'جيتيت']
20
3


In [36]:
wordKU = knownUnknownWords[3]
wordKUtokens = tokenizer.tokenize(wordKU)
print(wordKU)
print(wordKUtokens)
print()

any_true_root, all_true_root, is_roots, roots, real_roots =check_for_root(wordKUtokens)

for i in range(len(is_roots)):
    is_root = is_roots[i]
    
    print("Word Token:", wordKUtokens[i], "\nRoot word:", roots[i], "\nIn rootlist:", is_roots[i],
          "\n-------------------------------------")

بنجران
['بنج', '##ران']

Word Token: بنج 
Root word: نجج 
In rootlist: True 
-------------------------------------
Word Token: ##ران 
Root word: رون 
In rootlist: True 
-------------------------------------


In [37]:
parse_dotless_text("بعد المطر تأتي الشمس وتزهر الزهور")

'بعد_المطر_بابى_السمس_وبرهر_الرهور'

In [38]:


import tkinter as tk

second_probable_word = ""
cursor_position = 0

def function123(input_text, cursor_position):
    # Get the text up to the cursor position
    text_up_to_cursor = input_text[:cursor_position]

    # Split the text into words
    words = text_up_to_cursor.split()

    # The word number that the user is currently editing is the number of words
    word_number = len(words) - 1
    current_word = words[word_number]

    print("current_word:", current_word)
    print("Cursor position:", cursor_position)
    print("Word Number:", word_number)
    print("input_text:", input_text)

    Fexample = mask_word(input_text, word_number)
    Fword_probabilities, Fsorted_words, Fmost_probable_word = generate_probabilties_end(Fexample)
    Msentence = Fexample["Masked"]

    if Fmost_probable_word == None:
        fill = words[word_number]
    else:
        fill = Fmost_probable_word
        if len(Fsorted_words) > 1:
            firstWord = Fsorted_words[1]
        else:
            firstWord = ""
        if len(Fsorted_words) > 2:
            secondWord = Fsorted_words[2]
        else:
            secondWord = ""

    
    sentence = Msentence.replace("[MASK]", fill)

    print("example:", Fexample)
    print("most_probable_word:", Fmost_probable_word)
    print("sorted_words:", Fsorted_words)
    print("fill:", fill)
    print("sentence:", sentence)
    print("--------------------------------------------------")
    return sentence, firstWord, secondWord

def insert_text():
    text_to_insert = insert_button.cget("text")  
    print(text_to_insert)
    current_text = entry.get()
    words = current_text.split()
    words = words[:-1]
    new_text = ' '.join(words) + " " + text_to_insert
    print(new_text)
    entry.delete(0, tk.END)
    entry.insert(0, new_text)

def insert_text2():
    text_to_insert = insert_button2.cget("text")  
    print(text_to_insert)
    current_text = entry.get()
    words = current_text.split()
    words = words[:-1]
    new_text = ' '.join(words) + " " + text_to_insert
    print(new_text)
    entry.delete(0, tk.END)
    entry.insert(0, new_text)
    
def update_input(event):
    # Ignore non-character key presses
    if event.keysym == "BackSpace":
        pass  # Process backspace here if needed
    elif event.char == "" or event.keysym == "space":
        return

    # Get the current text input from the Entry widget
    input_text = entry.get()

    # Call the function123() with the current input and get the processed string
    cursor_position = entry.index(tk.INSERT)

    processed_text, wordOne, wordTwo = function123(input_text, cursor_position)
    print(wordOne, wordTwo)
    insert_button.config(text=wordOne)
    insert_button2.config(text=wordTwo)

    # Replace the text in the Entry widget with the processed string
    entry.delete(0, tk.END)
    entry.insert(0, processed_text)
    #entry.icursor(cursor_position)



# Create the main window
main = tk.Tk()
main.geometry("800x600")
# Create the Entry widget
entry_font = ("Helvetica", 50)  # Adjust the font family and size as needed
entry = tk.Entry(main, font=entry_font)

# Adjust entry box size with the width option
entry_width = 60  # Adjust the width as needed
entry.config(width=entry_width)

# Bind the <Key> event to the update_input function

entry.bind("<KeyRelease>", update_input)

# Pack the Entry widget to display it in the window
entry.pack()

button_font = ("Helvetica", 20)  # Adjust the font family and size as needed

button_label = ""

insert_button = tk.Button(main, text=button_label, command=insert_text, font=button_font)
insert_button.config(width=25)
insert_button.pack()

button_label = ""
insert_button2 = tk.Button(main, text=button_label, command=insert_text2, font=button_font)
insert_button2.config(width=25)
insert_button2.pack()

# Start the main event loop
main.mainloop()

current_word: ا
Cursor position: 1
Word Number: 0
input_text: ا
example: {'Masked': '[MASK]', 'Options': ['ا', 'أ', 'إ', 'آ', 'ء'], 'Target': 'ا'}
most_probable_word: ا
sorted_words: ['ا', 'أ', 'إ', 'ء', 'آ']
fill: ا
sentence: ا
--------------------------------------------------
أ إ
current_word: ال
Cursor position: 2
Word Number: 0
input_text: ال
example: {'Masked': '[MASK]', 'Options': ['ءل', 'آل', 'إل', 'أل', 'ال'], 'Target': 'ال'}
most_probable_word: إل
sorted_words: ['إل', 'آل']
fill: إل
sentence: إل
--------------------------------------------------
آل 
current_word: إلس
Cursor position: 3
Word Number: 0
input_text: إلس
example: {'Masked': '[MASK]', 'Options': ['ءلس', 'ءلش', 'آلس', 'آلش', 'إلس', 'إلش', 'ألس', 'ألش', 'الس', 'الش'], 'Target': 'إلس'}
most_probable_word: الس
sorted_words: ['الس', 'ألس', 'الش']
fill: الس
sentence: الس
--------------------------------------------------
ألس الش
current_word: السم
Cursor position: 4
Word Number: 0
input_text: السم
example: {'Masked': '[M

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/ammar/Developer/git-repos/dotless/env/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/74/rsfrzb6x50l9hjwqyty2zw000000gn/T/ipykernel_11838/3750098262.py", line 85, in update_input
    processed_text, wordOne, wordTwo = function123(input_text, cursor_position)
                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/74/rsfrzb6x50l9hjwqyty2zw000000gn/T/ipykernel_11838/3750098262.py", line 48, in function123
    return sentence, firstWord, secondWord
           ^^^^^^^^
UnboundLocalError: cannot access local variable 'firstWord' where it is not associated with a value


current_word: الشمسوف
Cursor position: 7
Word Number: 0
input_text: الشمسوف
example: {'Masked': '[MASK]', 'Options': ['ءلشمشؤف', 'ءلشمشؤق', 'ءلشمشوف', 'ءلشمشوق', 'ءلشمسؤف', 'ءلشمسؤق', 'ءلشمسوف', 'ءلشمسوق', 'ءلسمشؤف', 'ءلسمشؤق', 'ءلسمشوف', 'ءلسمشوق', 'ءلسمسؤف', 'ءلسمسؤق', 'ءلسمسوف', 'ءلسمسوق', 'آلشمشؤف', 'آلشمشؤق', 'آلشمشوف', 'آلشمشوق', 'آلشمسؤف', 'آلشمسؤق', 'آلشمسوف', 'آلشمسوق', 'آلسمشؤف', 'آلسمشؤق', 'آلسمشوف', 'آلسمشوق', 'آلسمسؤف', 'آلسمسؤق', 'آلسمسوف', 'آلسمسوق', 'إلشمشؤف', 'إلشمشؤق', 'إلشمشوف', 'إلشمشوق', 'إلشمسؤف', 'إلشمسؤق', 'إلشمسوف', 'إلشمسوق', 'إلسمشؤف', 'إلسمشؤق', 'إلسمشوف', 'إلسمشوق', 'إلسمسؤف', 'إلسمسؤق', 'إلسمسوف', 'إلسمسوق', 'ألشمشؤف', 'ألشمشؤق', 'ألشمشوف', 'ألشمشوق', 'ألشمسؤف', 'ألشمسؤق', 'ألشمسوف', 'ألشمسوق', 'ألسمشؤف', 'ألسمشؤق', 'ألسمشوف', 'ألسمشوق', 'ألسمسؤف', 'ألسمسؤق', 'ألسمسوف', 'ألسمسوق', 'الشمشؤف', 'الشمشؤق', 'الشمشوف', 'الشمشوق', 'الشمسؤف', 'الشمسؤق', 'الشمسوف', 'الشمسوق', 'السمشؤف', 'السمشؤق', 'السمشوف', 'السمشوق', 'السمسؤف', 'السمسؤق', 'السمسوف', 'السمسوق'], '

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/ammar/Developer/git-repos/dotless/env/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/74/rsfrzb6x50l9hjwqyty2zw000000gn/T/ipykernel_11838/3750098262.py", line 85, in update_input
    processed_text, wordOne, wordTwo = function123(input_text, cursor_position)
                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/74/rsfrzb6x50l9hjwqyty2zw000000gn/T/ipykernel_11838/3750098262.py", line 48, in function123
    return sentence, firstWord, secondWord
           ^^^^^^^^
UnboundLocalError: cannot access local variable 'firstWord' where it is not associated with a value


current_word: الشمسوفم
Cursor position: 8
Word Number: 0
input_text: الشمسوفم
example: {'Masked': '[MASK]', 'Options': ['ءلشمشؤقم', 'ءلشمشؤفم', 'ءلشمشوقم', 'ءلشمشوفم', 'ءلشمسؤقم', 'ءلشمسؤفم', 'ءلشمسوقم', 'ءلشمسوفم', 'ءلسمشؤقم', 'ءلسمشؤفم', 'ءلسمشوقم', 'ءلسمشوفم', 'ءلسمسؤقم', 'ءلسمسؤفم', 'ءلسمسوقم', 'ءلسمسوفم', 'آلشمشؤقم', 'آلشمشؤفم', 'آلشمشوقم', 'آلشمشوفم', 'آلشمسؤقم', 'آلشمسؤفم', 'آلشمسوقم', 'آلشمسوفم', 'آلسمشؤقم', 'آلسمشؤفم', 'آلسمشوقم', 'آلسمشوفم', 'آلسمسؤقم', 'آلسمسؤفم', 'آلسمسوقم', 'آلسمسوفم', 'إلشمشؤقم', 'إلشمشؤفم', 'إلشمشوقم', 'إلشمشوفم', 'إلشمسؤقم', 'إلشمسؤفم', 'إلشمسوقم', 'إلشمسوفم', 'إلسمشؤقم', 'إلسمشؤفم', 'إلسمشوقم', 'إلسمشوفم', 'إلسمسؤقم', 'إلسمسؤفم', 'إلسمسوقم', 'إلسمسوفم', 'ألشمشؤقم', 'ألشمشؤفم', 'ألشمشوقم', 'ألشمشوفم', 'ألشمسؤقم', 'ألشمسؤفم', 'ألشمسوقم', 'ألشمسوفم', 'ألسمشؤقم', 'ألسمشؤفم', 'ألسمشوقم', 'ألسمشوفم', 'ألسمسؤقم', 'ألسمسؤفم', 'ألسمسوقم', 'ألسمسوفم', 'الشمشؤقم', 'الشمشؤفم', 'الشمشوقم', 'الشمشوفم', 'الشمسؤقم', 'الشمسؤفم', 'الشمسوقم', 'الشمسوفم', 'السمشؤقم', 'الس

...

current_word: وسهل
Cursor position: 9
Word Number: 1
input_text: أهلا وسهل
example: {'Masked': 'أهلا [MASK]', 'Options': ['ؤشةل', 'ؤشهل', 'ؤسةل', 'ؤسهل', 'وشةل', 'وشهل', 'وسةل', 'وسهل'], 'Target': 'وسهل'}
most_probable_word: وسهل
sorted_words: ['وسهل']
fill: وسهل
sentence: أهلا وسهل
--------------------------------------------------
 
current_word: وسهلا
Cursor position: 10
Word Number: 1
input_text: أهلا وسهلا
example: {'Masked': 'أهلا [MASK]', 'Options': ['ؤشةلا', 'ؤشةلأ', 'ؤشةلإ', 'ؤشةلآ', 'ؤشةلء', 'ؤشهلا', 'ؤشهلأ', 'ؤشهلإ', 'ؤشهلآ', 'ؤشهلء', 'ؤسةلا', 'ؤسةلأ', 'ؤسةلإ', 'ؤسةلآ', 'ؤسةلء', 'ؤسهلا', 'ؤسهلأ', 'ؤسهلإ', 'ؤسهلآ', 'ؤسهلء', 'وشةلا', 'وشةلأ', 'وشةلإ', 'وشةلآ', 'وشةلء', 'وشهلا', 'وشهلأ', 'وشهلإ', 'وشهلآ', 'وشهلء', 'وسةلا', 'وسةلأ', 'وسةلإ', 'وسةلآ', 'وسةلء', 'وسهلا', 'وسهلأ', 'وسهلإ', 'وسهلآ', 'وسهلء'], 'Target': 'وسهلا'}
most_probable_word: وسهلا
sorted_words: ['وسهلا']
fill: وسهلا
sentence: أهلا وسهلا
--------------------------------------------------
 
current_word: وسهلا
Cu

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/ammar/Developer/git-repos/dotless/env/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/74/rsfrzb6x50l9hjwqyty2zw000000gn/T/ipykernel_11838/3750098262.py", line 85, in update_input
    processed_text, wordOne, wordTwo = function123(input_text, cursor_position)
                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/74/rsfrzb6x50l9hjwqyty2zw000000gn/T/ipykernel_11838/3750098262.py", line 48, in function123
    return sentence, firstWord, secondWord
           ^^^^^^^^
UnboundLocalError: cannot access local variable 'firstWord' where it is not associated with a value


current_word: وس
Cursor position: 7
Word Number: 1
input_text: أهلا وس
example: {'Masked': 'أهلا [MASK]', 'Options': ['ؤس', 'ؤش', 'وس', 'وش'], 'Target': 'وس'}
most_probable_word: وس
sorted_words: ['وس', 'وش']
fill: وس
sentence: أهلا وس
--------------------------------------------------
وش 
current_word: و
Cursor position: 6
Word Number: 1
input_text: أهلا و
example: {'Masked': 'أهلا [MASK]', 'Options': ['و', 'ؤ'], 'Target': 'و'}
most_probable_word: و
sorted_words: ['و', 'ؤ']
fill: و
sentence: أهلا و
--------------------------------------------------
ؤ 
current_word: أهلا
Cursor position: 5
Word Number: 0
input_text: أهلا 
example: {'Masked': '[MASK]', 'Options': ['ءةلا', 'ءةلأ', 'ءةلإ', 'ءةلآ', 'ءةلء', 'ءهلا', 'ءهلأ', 'ءهلإ', 'ءهلآ', 'ءهلء', 'آةلا', 'آةلأ', 'آةلإ', 'آةلآ', 'آةلء', 'آهلا', 'آهلأ', 'آهلإ', 'آهلآ', 'آهلء', 'إةلا', 'إةلأ', 'إةلإ', 'إةلآ', 'إةلء', 'إهلا', 'إهلأ', 'إهلإ', 'إهلآ', 'إهلء', 'أةلا', 'أةلأ', 'أةلإ', 'أةلآ', 'أةلء', 'أهلا', 'أهلأ', 'أهلإ', 'أهلآ', 'أهلء', 'اةلا', 

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/ammar/Developer/git-repos/dotless/env/lib/python3.11/tkinter/__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "/var/folders/74/rsfrzb6x50l9hjwqyty2zw000000gn/T/ipykernel_11838/3750098262.py", line 85, in update_input
    processed_text, wordOne, wordTwo = function123(input_text, cursor_position)
                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/74/rsfrzb6x50l9hjwqyty2zw000000gn/T/ipykernel_11838/3750098262.py", line 15, in function123
    current_word = words[word_number]
                   ~~~~~^^^^^^^^^^^^^
IndexError: list index out of range


: 